# Text Classifiers

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
import ipynb
from ipynb.fs.defs.data_collect_preprocessing1 import get_x_y_preprocessed
import numpy as np

In [3]:
MY_EMBEDDING_PATH = "C:\\Users\\lpdepersiis\\PycharmProjects\\autoencoderNlp\\embedding\\en\\glove\\"


In [4]:
x_rec, y_rec = get_x_y_preprocessed("dataset_cat")

In [5]:
print(y_rec.shape)

(3449, 7)


In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts(x_rec)

In [8]:
word_index = tokenizer.word_index

In [9]:
word_index["sport"]

23

In [10]:
print(type(word_index))

<class 'dict'>


In [11]:
print(len(word_index.keys()))

40977


In [12]:
def get_word_matrix():
    """
    Questa funzione ci serve per crearci un dizionario avente come indice la parola e come valore il vettore dell'embedding corrispondente
    """
    word_matrix = {}
    with open(MY_EMBEDDING_PATH + 'glove.6B.100d.txt', 'r', encoding='UTF-8') as file_emb:
        for row in file_emb: # leggo ogni riga del file di testo contenente l'embedding
            row = row.split() # la divido nei suoi elementi
            word_matrix[row[0]] = np.array(row[1:], dtype='float32') # il primo è la parola e sarà l'indice di questa voce, gli altri andranno a formare il vettore 
    return word_matrix

##### proviamo se il dizionario è come ci aspettiamo

In [13]:
word_matrix = get_word_matrix()


In [14]:
word_matrix["house"]  

array([-0.18867  , -0.040943 ,  0.58878  ,  0.11062  ,  0.14236  ,
        0.4885   , -0.31832  ,  0.53819  , -0.018549 ,  0.029687 ,
        0.30299  , -0.16522  , -0.18896  ,  0.5148   , -0.79405  ,
        0.26409  ,  0.027747 ,  0.041163 , -0.49378  , -0.14263  ,
        0.29017  , -0.25369  ,  0.70559  , -1.0501   , -0.49344  ,
       -0.37148  , -0.85796  , -0.55158  , -0.60251  , -0.0099676,
        0.8725   ,  0.12149  ,  0.551    ,  0.49924  , -0.3088   ,
        1.1067   , -0.15494  , -0.29923  ,  0.91149  ,  0.19859  ,
       -0.73946  , -1.0182   ,  0.37208  , -0.10043  ,  0.13537  ,
       -0.52687  , -0.60437  , -0.15906  ,  0.49283  , -0.61386  ,
        0.046815 , -0.88806  ,  0.60229  ,  0.72199  , -0.4316   ,
       -3.0706   , -0.11233  , -0.45713  ,  0.95737  ,  0.59174  ,
       -0.17124  ,  0.65746  ,  0.44741  ,  0.6101   ,  1.0216   ,
       -0.2458   ,  0.90191  ,  0.78319  ,  0.28272  , -0.4539   ,
        0.16309  , -0.0078932, -0.27714  , -0.87249  , -0.1971

##### ora abbiamo l'indice ottenuto tramite Tokenizer dai nostri testi ed abbiamo un dizionario che fa corrispondere ad ogni parola il vettore corrispondente dell'embedding GloVe, dobbiamo creare una matrice in cui le righe siano nello stesso ordine dell'indice ottenuto dal tokenizer e che contenga solo quei vettori (più lo 0 che è lasciato con un vettore di zeri per essere usato ad indicare l'assenza di una parola)

###### aggiungo qui la costante relativa alla dimensione dei vettori dell'embedding, perché non si perda la consequenzialità, ma andrebbe messa sopra insieme alle altre variabili (la stessa già impostata nell'altro file)

In [15]:
EMBEDDING_DIM = 100

##### Recuperiamo anche la costante relativa alla lunghezza delle frasi

In [16]:
SENTENCE_LENGTH = 80

In [18]:
def get_embedding_matrix(embeddings_index, word_index, dim_embeddings=EMBEDDING_DIM):
    """
    Tramite questa funzione creiamo una matrice in cui le righe siano nello stesso ordine dell'indice ottenuto dal tokenizer 
    e che contenga solo i vettori relativi alle parole in esso contenute
    
    :param embeddings_index: il dizionario, ottenuto dall'embedding, avente le parole come indice ed i vettori come valore
    :param word_index:  il dizionario ottenuto dal tokenizer avente come indice la parola e come valore il suo indice
    :param dim_embeddings: la lunghezza dei vettori dell'embedding che stiamo utilizzando
    :return: la matrice dei vettori dell'embedding ordinata come il nostro indice

    """
    embedding_matrix = np.zeros((len(word_index) + 1, dim_embeddings))  # creiamo la matrice di zeri avente tante righe quante sono le parole (più una) e tante colonne quante sono quelle dei vettori
    print(embedding_matrix.shape)
    for word in word_index.keys():  # Scorriamo le parole dell'indice del tokenizer
        embedding_vector = embeddings_index.get(word)  # estraiamo il vettore corrispondente
        if embedding_vector is not None:  # verifichiamo che esista (anche se il nostro dizionario è più piccolo di quello dell'embedding potrebbe contenere parolo non presenti in esso)
            # se la parola è presente andiamo avanti (se non è presente, in corrispondenza di questo indice, rimarrà il vettore formato da zeri)
            embedding_matrix[word_index[word]] = embedding_vector  # impostiamo nella matrice quella riga con il vettore corrispondente alla parola
    
    return embedding_matrix

###### ora abbiamo tutti gli elementi per fare lo strato di tipo Embedding che sarà il primo della nostra rete, possiamo scrivere una funzione che lo valorizzi nel modo corretto


In [19]:
embedding_matrix = get_embedding_matrix(word_matrix, tokenizer.word_index)

(40978, 100)


In [20]:
def get_embedding_layer(embedding_matrix, input_length=SENTENCE_LENGTH, trainable=False):
    """
     Instanzia lo strato di tipo Embedding

    :param embedding_matrix: Il dizionario ottenuto dall'embedding avente le parole come indice e il vettore come valore
    :param input_length: La lunghezza delle frasi che saranno passate come input
    
    :return: lo strato di tipo Embedding
    """

    embedding_layer = Embedding(  # Creiamo un'istanza del layer di tipo Embedding ed impostiamo i parametri indispensabili e quelli necessari per le nostre esigenze
                        embedding_matrix.shape[0],  # il numero di righe (numero di parole + 1)
                        embedding_matrix.shape[1],  #  il numero di colonne (lunghezza dei vettori)
                        weights=[embedding_matrix],  # l'embedding_matrix creata tramite la funzione precedente
                        input_length=input_length,  # la lunghezza delle frasi
                        trainable=trainable)  # Impostiamo se questo strato deve essere addestrabile o meno, se lo impostiamo addestrabile i vettori si modificheranno

    return embedding_layer  # restituiamo lo strato Embedding



In [21]:
print(embedding_matrix.shape)

(40978, 100)


##### Ora riprendiamo la pipeline per la realizzazione del classificatore
##### trasformiamo la lista di frasi, ciascuna costituita da liste di parole, in liste degli indici corrispondenti

In [22]:
sequences = tokenizer.texts_to_sequences(x_rec)

In [24]:
print(len(sequences))

3449


In [25]:
print(x_rec[0])

['rowing,', 'sometimes', 'referred', 'crew', 'united', 'states,', 'sport', 'whose', 'origins', 'reach', 'back', 'ancient', 'egyptian', 'timesit', 'involves', 'propelling', 'boat', '(racing', 'shell)', 'water', 'using', 'oarsby', 'pushing', 'water', 'oars,', 'rowers', 'generate', 'force', 'move', 'boatthe', 'sport', 'either', 'recreational', 'enjoyment', 'fitness,', 'competitive,', 'athletes', 'race', 'one', 'another', 'boatsthe', 'training', 'physical', 'strain', 'body', 'required', 'successful', 'rower', 'intense']


##### al posto delle parole della frase precedente sono stati messi gli indici corrispondenti

In [26]:
print(sequences[0])

[2901, 206, 312, 1575, 51, 287, 23, 476, 1090, 1638, 332, 98, 2665, 9288, 465, 14330, 351, 14331, 14332, 408, 101, 14333, 7116, 408, 7117, 1283, 2666, 726, 1639, 7118, 23, 237, 2667, 3856, 7119, 9289, 127, 368, 7, 183, 14334, 477, 131, 4911, 210, 313, 758, 1284, 2668]


##### Facciamo il padding per portare tutte le sequenze alla lunghezza impostate con la costante SENTENCE_LENGTH 

##### Per farlo dobbiamo importare pad_sequences da keras

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [28]:
x_num_fix = pad_sequences(sequences, maxlen=SENTENCE_LENGTH)  


##### Infatti la sequence precedente è diventata:

In [29]:
x_num_fix[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,  2901,   206,   312,  1575,    51,
         287,    23,   476,  1090,  1638,   332,    98,  2665,  9288,
         465, 14330,   351, 14331, 14332,   408,   101, 14333,  7116,
         408,  7117,  1283,  2666,   726,  1639,  7118,    23,   237,
        2667,  3856,  7119,  9289,   127,   368,     7,   183, 14334,
         477,   131,  4911,   210,   313,   758,  1284,  2668])

##### Creiamo il modello, prima di farlo dobbiamo importare i layer necessari 

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout


In [31]:
embedding_layer = get_embedding_layer(embedding_matrix)

##### Prima di lanciare l'addestramento dividiamo il dataset tra una parte da utilizzare per il train ed una per la validazione  
##### Per farlo utilizziamo una funzione di sklearn: train_test_split

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(x_num_fix, y_rec, test_size=0.20, random_state=42)  # dividiamo il dataset lasciando una parte (0.2 quindi 20%) per la validation


In [49]:
model = Sequential()  # Instanziamo Sequential
model.add(Embedding(embedding_matrix.shape[0],  # il numero di righe (numero di parole + 1)
                        embedding_matrix.shape[1],  #  il numero di colonne (lunghezza dei vettori)
                        input_length=SENTENCE_LENGTH,  # la lunghezza delle frasi
                        trainable=True))
model.add(Dense(128, name="Layer1"))  # Aggiungiamo i vari layers (gli assegnamo anche un nome per individuarli nella stampa successiva)
model.add(Dense(64, name="Layer2")) 
model.add(Flatten(name="No_Layer"))  # Aggiungiamo questo per passare dalle due dimensioni avute finora alla dimensione singola
model.add(Dense(64, name="Layer3"))
model.add(Dense(48, name="Layer_n-1"))
model.add(Dense(7, activation='softmax', name="Output_Layer"))  # Lo strato finale ha un numero di neuroni pari al numero di categorie
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])  # Compiliamo il modello definendo loss, metrica per la valutazione (accuratezza) 
print(model.summary())


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 80, 100)           4097800   
                                                                 
 Layer1 (Dense)              (None, 80, 128)           12928     
                                                                 
 Layer2 (Dense)              (None, 80, 64)            8256      
                                                                 
 No_Layer (Flatten)          (None, 5120)              0         
                                                                 
 Layer3 (Dense)              (None, 64)                327744    
                                                                 
 Layer_n-1 (Dense)           (None, 48)                3120      
                                                                 
 Output_Layer (Dense)        (None, 7)                

In [50]:
history = model.fit(X_train, y_train, epochs=10, verbose=1, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
22/22 [==============================] - 2s 45ms/step - loss: 1.8930 - acc: 0.2341 - val_loss: 1.8332 - val_acc: 0.2609
Epoch 2/10
22/22 [==============================] - 1s 26ms/step - loss: 0.9188 - acc: 0.8300 - val_loss: 0.8504 - val_acc: 0.7000
Epoch 3/10
22/22 [==============================] - 1s 27ms/step - loss: 0.0478 - acc: 0.9931 - val_loss: 0.6609 - val_acc: 0.7855
Epoch 4/10
22/22 [==============================] - 1s 26ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.7254 - val_acc: 0.7971
Epoch 5/10
22/22 [==============================] - 1s 26ms/step - loss: 4.2660e-04 - acc: 1.0000 - val_loss: 0.7171 - val_acc: 0.8101
Epoch 6/10
22/22 [==============================] - 1s 26ms/step - loss: 1.7924e-04 - acc: 1.0000 - val_loss: 0.7158 - val_acc: 0.8072
Epoch 7/10
22/22 [==============================] - 1s 26ms/step - loss: 1.2501e-04 - acc: 1.0000 - val_loss: 0.7094 - val_acc: 0.8029
Epoch 8/10
22/22 [==============================] - 1s 26ms/step - loss

In [ ]:
model = Sequential()  # Instanziamo Sequential
model.add(embedding_layer)  # Aggiungiamo lo strato Embeddings appena creato
model.add(Dense(128, name="Layer1"))  # Aggiungiamo i vari layers (gli assegnamo anche un nome per individuarli nella stampa successiva)
model.add(Dense(64, name="Layer2")) 
model.add(Flatten(name="No_Layer"))  # Aggiungiamo questo per passare dalle due dimensioni avute finora alla dimensione singola
model.add(Dense(64, name="Layer3"))
model.add(Dense(48, name="Layer_n-1"))
model.add(Dense(7, activation='softmax', name="Output_Layer"))  # Lo strato finale ha un numero di neuroni pari al numero di categorie
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])  # Compiliamo il modello definendo loss, metrica per la valutazione (accuratezza) 
print(model.summary())

#### Lanciamo l'addestramento 

In [37]:
history = model.fit(X_train, y_train, epochs=10, verbose=1, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
22/22 [==============================] - 2s 33ms/step - loss: 1.3545 - acc: 0.5205 - val_loss: 0.8060 - val_acc: 0.7087
Epoch 2/10
22/22 [==============================] - 0s 12ms/step - loss: 0.4025 - acc: 0.8681 - val_loss: 0.6527 - val_acc: 0.7667
Epoch 3/10
22/22 [==============================] - 0s 12ms/step - loss: 0.1675 - acc: 0.9638 - val_loss: 0.6783 - val_acc: 0.7797
Epoch 4/10
22/22 [==============================] - 0s 12ms/step - loss: 0.0664 - acc: 0.9935 - val_loss: 0.7564 - val_acc: 0.7667
Epoch 5/10
22/22 [==============================] - 0s 12ms/step - loss: 0.0266 - acc: 1.0000 - val_loss: 0.8034 - val_acc: 0.7696
Epoch 6/10
22/22 [==============================] - 0s 12ms/step - loss: 0.0134 - acc: 1.0000 - val_loss: 0.8489 - val_acc: 0.7710
Epoch 7/10
22/22 [==============================] - 0s 12ms/step - loss: 0.0081 - acc: 1.0000 - val_loss: 0.8875 - val_acc: 0.7696
Epoch 8/10
22/22 [==============================] - 0s 13ms/step - loss: 0.0056 - a

#### Notando che va molto presto in overfitting, è chiaro che dovremmo aumentare il numero degli esempi andando a raccogliere altri documenti, comunque possiamo provare ad aggiungere un po' di dropout

In [ ]:
model = Sequential()  
model.add(embedding_layer) 
model.add(Dense(128, name="Layer1"))  
model.add(Dropout(0.5)) # Strato di Dropout che ad ogni passaggio disabilita una certa quantità di connessioni (in questo caso il 30%)
model.add(Dense(64, name="Layer2"))  
model.add(Flatten(name="No_Layer"))
model.add(Dropout(0.4))
model.add(Dense(64, name="Layer3"))
model.add(Dropout(0.3)) # Qui ne disabilitiamo il 20%
model.add(Dense(48, name="Layer_n-1"))
model.add(Dense(7, activation='softmax', name="Output_Layer"))  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])  
print(model.summary())


In [39]:
history = model.fit(X_train, y_train, epochs=10, verbose=1, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
22/22 [==============================] - 2s 38ms/step - loss: 2.1841 - acc: 0.3298 - val_loss: 0.9979 - val_acc: 0.6594
Epoch 2/10
22/22 [==============================] - 0s 15ms/step - loss: 1.0874 - acc: 0.6173 - val_loss: 0.8194 - val_acc: 0.7087
Epoch 3/10
22/22 [==============================] - 0s 16ms/step - loss: 0.7882 - acc: 0.7177 - val_loss: 0.6840 - val_acc: 0.7580
Epoch 4/10
22/22 [==============================] - 0s 16ms/step - loss: 0.6204 - acc: 0.7767 - val_loss: 0.6423 - val_acc: 0.7754
Epoch 5/10
22/22 [==============================] - 0s 17ms/step - loss: 0.5310 - acc: 0.8112 - val_loss: 0.6232 - val_acc: 0.7710
Epoch 6/10
22/22 [==============================] - 0s 16ms/step - loss: 0.4506 - acc: 0.8267 - val_loss: 0.6334 - val_acc: 0.7812
Epoch 7/10
22/22 [==============================] - 0s 17ms/step - loss: 0.4242 - acc: 0.8449 - val_loss: 0.6150 - val_acc: 0.7841
Epoch 8/10
22/22 [==============================] - 0s 17ms/step - loss: 0.3734 - a

### A questo punto si posso provare altri aggiustamenti, tra cui soprattutto aggiungere strati più sofisticati che prevedono convoluzione e ricorrenza
### Ad esempio Conv1D, Bidirectional, GRU, LSTM ecc.

In [41]:
from tensorflow.keras.layers import Conv1D, Bidirectional, GRU, LSTM


In [42]:
model = Sequential() 
model.add(embedding_layer)  
model.add(Bidirectional(GRU(64)))
model.add(Dense(128, name="Layer1"))  
model.add(Dropout(0.4))
model.add(Dense(64, name="Layer2")) 
model.add(Flatten(name="No_Layer")) 
model.add(Dense(64, name="Layer3"))
model.add(Dropout(0.3))
model.add(Dense(48, name="Layer_n-1"))
model.add(Dense(7, activation='softmax', name="Output_Layer"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc']) 
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 100)           4097800   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              63744     
 l)                                                              
                                                                 
 Layer1 (Dense)              (None, 128)               16512     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 Layer2 (Dense)              (None, 64)                8256      
                                                                 
 No_Layer (Flatten)          (None, 64)                0         
                                                      

In [43]:
history = model.fit(X_train, y_train, epochs=15, verbose=1, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/15
22/22 [==============================] - 35s 105ms/step - loss: 1.7917 - acc: 0.2878 - val_loss: 1.4954 - val_acc: 0.4696
Epoch 2/15
22/22 [==============================] - 1s 35ms/step - loss: 1.4541 - acc: 0.4715 - val_loss: 1.2843 - val_acc: 0.5333
Epoch 3/15
22/22 [==============================] - 1s 36ms/step - loss: 1.2303 - acc: 0.5611 - val_loss: 1.0814 - val_acc: 0.6217
Epoch 4/15
22/22 [==============================] - 1s 37ms/step - loss: 0.9986 - acc: 0.6419 - val_loss: 0.8649 - val_acc: 0.6855
Epoch 5/15
22/22 [==============================] - 1s 37ms/step - loss: 0.7849 - acc: 0.7264 - val_loss: 0.7220 - val_acc: 0.7420
Epoch 6/15
22/22 [==============================] - 1s 38ms/step - loss: 0.6232 - acc: 0.7880 - val_loss: 0.5863 - val_acc: 0.7942
Epoch 7/15
22/22 [==============================] - 1s 35ms/step - loss: 0.5261 - acc: 0.8112 - val_loss: 0.5464 - val_acc: 0.8116
Epoch 8/15
22/22 [==============================] - 1s 36ms/step - loss: 0.4706 -

#### L'accuratezza del dataset di validazione è decisamente maggiore ed abbiamo ancora margine avendo una loss ancora alta e l'accuratezza del train set ancora sotto l'1
### Lanciamo nuovamente l'addestramento facendo altre 9 epoche

In [44]:
history = model.fit(X_train, y_train, epochs=9, verbose=1, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/9
22/22 [==============================] - 1s 43ms/step - loss: 0.1976 - acc: 0.9326 - val_loss: 0.4910 - val_acc: 0.8652
Epoch 2/9
22/22 [==============================] - 1s 36ms/step - loss: 0.1759 - acc: 0.9377 - val_loss: 0.5181 - val_acc: 0.8652
Epoch 3/9
22/22 [==============================] - 1s 35ms/step - loss: 0.1440 - acc: 0.9478 - val_loss: 0.5139 - val_acc: 0.8652
Epoch 4/9
22/22 [==============================] - 1s 35ms/step - loss: 0.1213 - acc: 0.9558 - val_loss: 0.6013 - val_acc: 0.8522
Epoch 5/9
22/22 [==============================] - 1s 38ms/step - loss: 0.1585 - acc: 0.9445 - val_loss: 0.6240 - val_acc: 0.8594
Epoch 6/9
22/22 [==============================] - 1s 36ms/step - loss: 0.1297 - acc: 0.9529 - val_loss: 0.6010 - val_acc: 0.8681
Epoch 7/9
22/22 [==============================] - 1s 36ms/step - loss: 0.1106 - acc: 0.9572 - val_loss: 0.6039 - val_acc: 0.8565
Epoch 8/9
22/22 [==============================] - 1s 37ms/step - loss: 0.1250 - acc: 0.95

#### Proviamo il modello con delle frasi nuove

In [45]:
from ipynb.fs.defs.data_collect_preprocessing import preprocess_text
from ipynb.fs.defs.data_collect_preprocessing import CATEGORIES

In [46]:
text = "There are two broad stances about what is the world studied by metaphysics. The strong, classical view assumes that the objects studied by metaphysics exist independently of any observer so that the subject is the most fundamental of all sciences."
text = preprocess_text(text)
sequence = tokenizer.texts_to_sequences([text])
print(sequence)
padded_seq = pad_sequences(sequence, maxlen=SENTENCE_LENGTH) 
y = model.predict(padded_seq)
print(y)
print(np.argmax(y))
print(CATEGORIES[np.argmax(y)])

[[24, 909, 16901, 15, 580, 18113, 138, 219, 2421, 425, 580, 1413, 787, 2073, 8182, 385, 502]]
[[0.02623279 0.00159637 0.00317192 0.17968048 0.62699157 0.00201284
  0.16031402]]
4
philosophy
